In [32]:
import requests
import pandas as pd

# Define API headers and base URL
API_TOKEN = "dlWWGBsFlpygEYosHpgYpxInxBBJeWse"
HEADERS = {"token": API_TOKEN}
BASE_URL = "https://www.ncei.noaa.gov/cdo-web/api/v2/"

In [33]:
def fetch_climate_data_with_tmax(dataset_id, location_id, start_date, end_date):
    url = f"{BASE_URL}data"
    params = {
        "datasetid": dataset_id,
        "locationid": location_id,
        "startdate": start_date,
        "enddate": end_date,
        "datatypeid": ["TMAX", "PRCP"],  # Fetch max temperature and precipitation
        "units": "metric",
        "limit": 1000
    }

    response = requests.get(url, headers=HEADERS, params=params)
    if response.status_code == 200:
        data = response.json().get("results", [])
        if data:
            return pd.DataFrame(data)
        else:
            print("No data found for the given parameters.")
            return pd.DataFrame()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return pd.DataFrame()

In [34]:
# Example: Fetching daily summaries for a specific location
dataset_id = "GHCND"  # Daily Summaries
location_id = "FIPS:37"  # Example: North Carolina (FIPS Code)
start_date = "2023-12-01"
end_date = "2023-12-31"

In [35]:
# Fetch and process the data
climate_data = fetch_climate_data_with_tmax(dataset_id, location_id, start_date, end_date)
print(climate_data.head())

                  date datatype            station attributes  value
0  2023-12-01T00:00:00     PRCP  GHCND:US1NCAG0001   ,,N,1000    0.3
1  2023-12-01T00:00:00     PRCP  GHCND:US1NCAG0007  T,,N,0700    0.0
2  2023-12-01T00:00:00     PRCP  GHCND:US1NCAG0008   ,,N,0700    0.0
3  2023-12-01T00:00:00     PRCP  GHCND:US1NCAL0036   ,,N,0800    0.0
4  2023-12-01T00:00:00     PRCP  GHCND:US1NCAL0039   ,,N,0700    0.0


In [36]:
# Assuming 'datatype' column contains metric types like PRCP (precipitation), TMAX (max temp), etc.
if not climate_data.empty:
    # Extract relevant metrics
    climate_data['date'] = pd.to_datetime(climate_data['date'])
    metrics = climate_data.pivot_table(
        index='date', columns='datatype', values='value', aggfunc='mean'
    )
    metrics = metrics.resample('M').mean()  # Monthly averages
    metrics.reset_index(inplace=True)

    print("Processed Climate Metrics:")
    print(metrics.head())
else:
    print("No climate data fetched.")


Processed Climate Metrics:
datatype       date      PRCP       TMAX
0        2023-12-31  0.150543  15.345614


In [37]:
# Example: Merge project data with climate metrics
projects = pd.DataFrame({
    'project_id': [1, 2, 3],
    'budget': [50000, 75000, 100000],
    'risk': [0.2, 0.5, 0.3],
    'ESG_base_score': [70, 85, 90]
})

# Add climate impact scores (example: average temperature, precipitation weightings)
projects['avg_temperature'] = metrics['TMAX'].mean()  # Example: average max temp
projects['precipitation'] = metrics['PRCP'].mean()  # Example: average precipitation

# Compute final ESG score with climate metrics
projects['final_ESG_score'] = (
    0.7 * projects['ESG_base_score'] + 
    0.2 * projects['avg_temperature'] - 
    0.1 * projects['precipitation']
)
print("Projects with Final ESG Scores:")
print(projects)


Projects with Final ESG Scores:
   project_id  budget  risk  ESG_base_score  avg_temperature  precipitation  \
0           1   50000   0.2              70        15.345614       0.150543   
1           2   75000   0.5              85        15.345614       0.150543   
2           3  100000   0.3              90        15.345614       0.150543   

   final_ESG_score  
0        52.054069  
1        62.554069  
2        66.054069  


In [38]:
from scipy.optimize import linprog

# Define optimization parameters
c = -projects['final_ESG_score']  # Negative because we're maximizing ESG impact
A = [projects['budget']]  # Budget constraints
b = [150000]  # Total budget available
bounds = [(0, 1) for _ in range(len(projects))]  # Allocation can be 0 or 1

# Solve the optimization problem
result = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')

# Interpret results
projects['selected'] = result.x.round()
print("Optimized Project Selection:")
print(projects[projects['selected'] == 1])


Optimized Project Selection:
   project_id  budget  risk  ESG_base_score  avg_temperature  precipitation  \
0           1   50000   0.2              70        15.345614       0.150543   
1           2   75000   0.5              85        15.345614       0.150543   

   final_ESG_score  selected  
0        52.054069       1.0  
1        62.554069       1.0  
